# < Contents Type Predict Modeling >

In [ ]:
# 경고 메시지 무시
import warnings
warnings.filterwarnings("ignore")

## 1. 학습 데이터 로드

In [ ]:
import pandas as pd

# 데이터 로드
raw = pd.read_excel('data/학습데이터_v1.xlsx')

# 데이터 확인
raw[:3]

,keyword,site,date,title,username,content,click,link,type
0,릴리바이레드 무스틴트,naverBlog,2019-08-09,더블유랩 셀피립틴트! 부드러운 무스제형 벨벳틴트 장단점 확실하게 리뷰했어요,아영,안녕하세요! 뷰스타 아영입니다:)최근에 립제품이 정말 다양해지는 거 같더라구요!뻑뻑...,NaN,https://m.blog.naver.com/loveneet?Redirect=Log...,바이럴
1,릴리바이레드 무스틴트,naverBlog,2019-08-10,센카 클렌징티슈 후기,꽃사슴 소녀,안녕하세요!꽃사슴소녀입니다​​요즘같은 시국에일본제품을 리뷰한다는것이초콤 마음에 걸리...,NaN,https://m.blog.naver.com/fairyasrai?Redirect=L...,바이럴
2,릴리바이레드 무스틴트,naverBlog,2019-08-11,[Y.S.L] 입생로랑 틴트 후기 / 입생로랑 416 싸이키델릭칠리 / 입생로랑 바...,설연이네,#입생로랑#입생로랑416#입생로랑바이닐크림틴트생일선물로 받은 #입생로랑틴트백화점 틴...,NaN,https://m.blog.naver.com/sy170416?Redirect=Log...,바이럴


In [ ]:
# 데이터 선택
data = raw[['content','type']]

# 데이터 확인
pd.value_counts(data['type'])

소비자    98
바이럴    71
Name: type, dtype: int64

## 2. 텍스트 데이터 정제

In [ ]:
import re

# 텍스트 정제 함수
def preprocessing_text(data):
    url = "(https?://)?(www.)?[a-zA-Z0-9./?=&-_]+[.]?[/][a-zA-Z0-9./?=&-_%]+"
    email = "[-_.+a-zA-Z0-9]+[@].+[.][[a-zA-Z0-9]+|[a-zA-Z0-9]+[.][a-zA-Z0-9]+]"
    
    for i in range(len(data)):
        data[i] = re.sub(url,"", data[i])
        data[i] = re.sub(email,"", data[i])
        data[i] = re.sub("[^가-힣a-zA-Z ]","", data[i])
        data[i] = data[i].lower()
    return data

# 텍스트 정제
data['content'] = preprocessing_text(data["content"].astype('str'))
data['char_len'] = [len(data['content'][_]) for _ in range(len(data['content']))]
data = data.loc[data['char_len']>5].reset_index(drop=True)
data[:3]

,content,type,char_len
0,안녕하세요 뷰스타 아영입니다최근에 립제품이 정말 다양해지는 거 같더라구요뻑뻑한 립제...,바이럴,1032
1,안녕하세요꽃사슴소녀입니다요즘같은 시국에일본제품을 리뷰한다는것이초콤 마음에 걸리지만이...,바이럴,1133
2,입생로랑입생로랑입생로랑바이닐크림틴트생일선물로 받은 입생로랑틴트백화점 틴트를 잘 접하...,바이럴,757


## 3. 형태소 분석 _Mecab

In [ ]:
# 형태소 분석기 Mecab
from konlpy.tag import Mecab
mecab = Mecab()

# mecab.morphs(clean_text[0])

# 명사만 추출
# mecab.nouns(data)

# 품사 포함 추출
# mecab.pos(clean_text[0],flatten=True)

# 형태소 분석
from tqdm import tqdm

corpus = []

for i in tqdm(range(len(data['content']))):
    sen = mecab.nouns(data['content'][i])
    sen = [word for word in sen if len(word) > 1] #1글자 이상만 추출
    corpus.append(sen)

100%|██████████| 169/169 [00:00<00:00, 201.47it/s]


## 4. 단어 정제

In [ ]:
# 바꾸기
clean_corpus = [[re.sub('^리바이$|^레드$','릴리바이레드',noun_) for noun_ in nouns] for nouns in corpus]

# 제거
clean_corpus = [[noun_ for noun_ in nouns if noun_ != "영화"] for nouns in clean_corpus]
clean_corpus = [[noun_ for noun_ in nouns if noun_ != "관람객"] for nouns in clean_corpus]

# 불용어 삭제
f = open("data/stopwords_list.txt", 'r', encoding='CP949')
line = [line.rstrip() for line in f.readlines()]

for i in range(len(line)):
    clean_corpus = [[noun_ for noun_ in nouns if noun_ != line[i]] for nouns in clean_corpus]

## 5. 단어 빈도 수

In [ ]:
# 단어 카운트 함수
def count_noun(data):
    t_noun = list()
    
    for _ in data:
        t_noun.extend(_)
    
    t_noun_s = pd.DataFrame(t_noun, columns=['word'])
    t_noun_f = t_noun_s.groupby('word').size().reset_index(name='count').sort_values(['count'], ascending=False).reset_index(drop=True)
    
    return t_noun_f

# 단어 카운트
frq_corpus = count_noun(clean_corpus)
frq_corpus[:10]

,word,count
0,릴리바이레드,1484
1,틴트,1191
2,컬러,941
3,제품,727
4,사용,397
5,무스,394
6,벨벳,345
7,발색,340
8,느낌,338
9,추천,337


## 6. TF-IDF

In [ ]:
for _ in range(len(clean_corpus)):
    clean_corpus[_] = re.sub("\[|\]","",str(clean_corpus[_]))
clean_corpus[0]

"'뷰스타', '아영', '최근', '제품', '다양', '제품', '제품', '더블유', '피립', '틴트', '무스', '제형', '틴트', '소개', '더블유', '피립', '틴트', '가격', '머랭', '질감', '틴트', '더블유', '피립', '틴트', '가지', '색상', '가지', '색상', '사용', '가지', '색상', '사용', '릴리바이레드', '코랄', '핑크', '핑크', '다양', '핑크', '제품', '더블유', '셀피', '틴트', '플리', '이터', '보통', '당량', '플리', '이터', '호수', '버건', '릴리바이레드', '릴리바이레드', '코랄', '컬러', '소프트', '홀릭', '로즈', '핑크', '색상', '스윗', '퍼플', '핑크', '색상', '포커스', '취향', '더블유', '피립', '틴트', '릴리바이레드', '릴리바이레드', '컬러', '무스', '제형', '각질', '부각', '제품', '정도', '코랄', '색상', '소프트', '홀릭', '색상', '각질', '부각', '주름', '세상', '분위기', '출근', '지속력', '색상', '요채', '컬러', '스윗', '구애', '일리', '핑크', '컬러', '조심', '색상', '제품', '요호', '포커스', '컬러', '친구', '진단', '컬러', '색상', '친구', '입술', '입술', '바이', '입술', '사람', '바이', '사람', '평소', '입술', '각질', '부각', '벨벳', '추천', '사용', '입술', '더블유', '제품', '제공', '작성'"

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model = TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9)

In [ ]:
tfidf_model.fit(clean_corpus)
tfidf_embedding = tfidf_model.transform(clean_corpus)
tfidf_embedding

<169x3678 sparse matrix of type '<class 'numpy.float64'>'
	with 27606 stored elements in Compressed Sparse Row format>

## 7. 모델링
### 1) 라벨 수치변환

In [ ]:
c_type = {"바이럴":1, "소비자":2}                                                                                                                               
data['type'] = data['type'].map(c_type)
data[:3]

,content,type,char_len
0,안녕하세요 뷰스타 아영입니다최근에 립제품이 정말 다양해지는 거 같더라구요뻑뻑한 립제...,1,1032
1,안녕하세요꽃사슴소녀입니다요즘같은 시국에일본제품을 리뷰한다는것이초콤 마음에 걸리지만이...,1,1133
2,입생로랑입생로랑입생로랑바이닐크림틴트생일선물로 받은 입생로랑틴트백화점 틴트를 잘 접하...,1,757


### 2) 단어 임베딩

In [ ]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_corpus)
sequences = tokenizer.texts_to_sequences(clean_corpus)

word_vocab = tokenizer.word_index

MAX_SEQUENCE_LENGTH = 424
word_vector = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post')
word_vector

array([[ 416, 3012,  287, ...,    0,    0,    0],
       [4408, 1060, 1389, ...,    0,    0,    0],
       [ 166,  166,  166, ...,    0,    0,    0],
       ...,
       [ 520,  156,  184, ...,    0,    0,    0],
       [7987, 7988, 7989, ...,    0,    0,    0],
       [1053,   45,   60, ...,    0,    0,    0]], dtype=int32)

In [ ]:
len(clean_corpus), len(data), len(word_vector)

(169, 169, 169)

### 3) 라벨 균형 맞추기
- Randomoversampler: 적은 수의 레이블 값을 복사  
(실무에서는 오버피팅 문제로 잘 활용하지 않음)  
  
- Smote: 군집 알고리즘을 이용하여 비슷한 데이터 생성  
(오버피팅 문제를 어느 정도 해결하여 실물에서 활용함)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=2020)
oversampled_data, oversampled_label = ros.fit_resample(word_vector, label)

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(k_neighbors=5, random_state=2020)
smoted_data, smoted_label = smote.fit_resample(tfidf_embedding, data['type'])

### 4) 머신러닝 알고리즘 학습

In [ ]:
from sklearn import svm, metrics, model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
# 훈련데이터와 테스트 데이터 나누기
train_data, test_data, train_label, test_label = train_test_split(smoted_data,smoted_label,random_state=0)
train_data[:4], train_label[:2]

(<4x3678 sparse matrix of type '<class 'numpy.float64'>'
 	with 1121 stored elements in Compressed Sparse Row format>,
 array([2, 1]))

#### GradientBossting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {"n_estimators":[1, 2, 3, 4, 5, 6,7,8,9,10], 
              "learning_rate":[0.001, 0.01, 0.1, 1, 10, 100],
             "max_depth":[1,2,3,4,5]}

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
Gr_clf = GradientBoostingClassifier()
Gr_clf.fit(train_data, train_label) #학습
predict = Gr_clf.predict(test_data) #예측
score = metrics.accuracy_score(test_label, predict)
print("정답률:", score)
# scores = model_selection.cross_val_score(GridSearchCV(Gr_clf, param_grid, cv=5), test_data, test_label, cv=10)
# print("교차검증 정답률 평균:", scores.mean())
# report = metrics.classification_report(test_label, predict)
# print(report)

정답률: 0.9795918367346939


In [ ]:
GB_grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5)
GB_grid.fit(train_data, train_label) #학습
predict = GB_grid.predict(test_data) #예측
score = metrics.accuracy_score(test_label, predict)
print("정답률:", score)
report = metrics.classification_report(test_label, predict)
print(report)

정답률: 0.9795918367346939
              precision    recall  f1-score   support

           1       1.00      0.96      0.98        24
           2       0.96      1.00      0.98        25

    accuracy                           0.98        49
   macro avg       0.98      0.98      0.98        49
weighted avg       0.98      0.98      0.98        49



In [ ]:
from sklearn.externals import joblib
joblib.dump(GB_grid, 'type_modeling.pkl')

['type_modeling.pkl']